# Classical Molecular Interaction Potentials tutorial using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of computing **classical molecular interaction potentials** from **protein structures**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **XXXX** protein (PDB code [XXXX](https://www.rcsb.org/structure/XXXX)).   
***

## Settings

### Biobb modules used

 - [biobb_cmip](https://github.com/bioexcel/biobb_cmip): Tools to compute classical molecular interaction potentials from protein structures.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [plotly](https://plotly.com/python/): Python Open Source Graphing Library. 


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_cmip.git
cd biobb_wf_cmip
conda env create -f conda_env/environment.yml
conda activate biobb_wf_cmip
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_cmip/notebooks/biobb_wf_cmip.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [PDB preparation (from PDB databank)](#preparePDB)
 3. [Molecular Interaction Potentials](#mips)
    1. [MIP+](#mip_pos) 
    2. [MIP-](#mip_neg) 
    3. [MIPn](#mip_neutral) 
 4. [PDB preparation (from MD)](#preparePDB_MD)
 5. [Interaction Potentials](#interaction)
    1. [Ligand](#ligand)
    2. [Protein](#protein)
 6. [Titration](#titration)
 7. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [ ]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

data_folder = "data/"
pdbCode = "1aki"
inputPDB = data_folder + pdbCode + ".pdb"

MDCode = "RBD-hACE2"
inputPDB_MD = data_folder + MDCode + ".noZN.pdb" 
inputTOP_MD = data_folder + MDCode + ".top" 

<a id="preparePDB"></a>
***
## PDB Preparation (from PDB databank)
CMIP tool needs additional information (e.g. charges, elements) included in the structure PDB file to properly run. In the next cells, a couple of BioBB building blocks will be used to prepare the input PDB file, adding this extra information.   
***
**Building Blocks** used:
 - [prepare_pdb](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_pdb) from **biobb_cmip.cmip.prepare_pdb**
***

In [ ]:
from biobb_cmip.cmip.prepare_pdb import prepare_pdb

cmipPDB = pdbCode + ".cmip.pdb"

prepare_pdb(input_pdb_path=inputPDB,
            output_cmip_pdb_path=cmipPDB)

<a id="mips"></a>
***
## Molecular Interaction Potentials (MIPs)
...   
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

In [ ]:
from biobb_cmip.cmip.cmip import cmip

mip_pos_log = pdbCode + ".mip_pos.log"
mip_pos_cube = pdbCode + ".mip_pos.cube"

prop = { 
    'execution_type' : 'mip_pos',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_pos_log,
     output_cube_path=mip_pos_cube,
     properties=prop)

In [ ]:
! cat 1aki.mip_pos.log

In [ ]:
view = nglview.show_structure_file(mip_pos_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="blue")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_cmip.cmip.cmip import cmip

mip_neg_log = pdbCode + ".mip_neg.log"
mip_neg_cube = pdbCode + ".mip_neg.cube"

prop = { 
    'execution_type' : 'mip_neg',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_neg_log,
     output_cube_path=mip_neg_cube,
     properties=prop)

In [ ]:
! cat 1aki.mip_neg.log

In [ ]:
view = nglview.show_structure_file(mip_neg_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="red")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_cmip.cmip.cmip import cmip

mip_neutral_log = pdbCode + ".mip_neutral.log"
mip_neutral_cube = pdbCode + ".mip_neutral.cube"

prop = { 
    'execution_type' : 'mip_neu',
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=mip_neutral_log,
     output_cube_path=mip_neutral_cube,
     properties=prop)

In [ ]:
view = nglview.show_structure_file(mip_neutral_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-1, color="grey")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
#Show different structures generated (for comparison)
view1 = nglview.show_structure_file(cmipPDB)
view1.add_component(mip_pos_cube)
view1.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view1.component_1.add_surface(isolevelType="value", isolevel=-5, color="blue")
view1.component_0.center()
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(cmipPDB)
view2.add_component(mip_neg_cube)
view2.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view2.component_1.add_surface(isolevelType="value", isolevel=-5, color="red")
view2.component_0.center()
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(cmipPDB)
view3.add_component(mip_neutral_cube)
view3.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view3.component_1.add_surface(isolevelType="value", isolevel=-1, color="grey")
view3.component_0.center()
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="preparePDB_MD"></a>
***
## PDB Preparation (from Molecular Dynamics)
...
***
**Building Blocks** used:
 - [prepare_structure](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_structure) from **biobb_cmip.cmip.prepare_structure**
***

In [ ]:
from biobb_structure_utils.utils.extract_chain import extract_chain

input_extract_chain = data_folder + "md.imaged.rot.dry.pdb"
inputPDB_MD_hACE2 = MDCode + ".hACE2.pdb"
inputPDB_MD_RBD = MDCode + ".RBD.pdb"

prop = {
    'chains': [ 'A' ],
    'permissive': True
}
extract_chain(input_structure_path=input_extract_chain,
              output_structure_path=inputPDB_MD_hACE2,
              properties=prop)

prop = {
    'chains': [ 'B' ],
    'permissive': True
}
extract_chain(input_structure_path=inputPDB_MD,
              output_structure_path=inputPDB_MD_RBD,
              properties=prop)

In [ ]:
from biobb_cmip.cmip.prepare_structure import prepare_structure

cmipPDB_MD_hACE2 = MDCode + ".hACE2.cmip.pdb"
cmipPDB_MD_RBD = MDCode + ".RBD.cmip.pdb"

prepare_structure(input_pdb_path=inputPDB_MD_hACE2,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=cmipPDB_MD_hACE2)

prepare_structure(input_pdb_path=inputPDB_MD_RBD,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=cmipPDB_MD_RBD)

<a id="interaction"></a>
***
## Interaction Potential Energies 
... 
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

In [ ]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "RBD.energies.log"
RBD_byat_out = "RBD.energies.byat.out"

prop = { 
    'execution_type' : 'energy',
    'remove_tmp': False
}

cmip(input_pdb_path=cmipPDB_MD_hACE2,
     input_probe_pdb_path=cmipPDB_MD_RBD,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     properties=prop)

In [ ]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(RBD_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat

atom_list, energy_dict = get_energies_byat(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [ ]:
from biobb_cmip.cmip.prepare_structure import prepare_structure


box_pdb_output = 'output_tit_all.pdb'

prepare_structure(input_pdb_path=inputPDB_MD,
                  input_topology_path=inputTOP_MD,
                  output_cmip_pdb_path=box_pdb_output)


In [ ]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "hACE2.box.log"
RBD_byat_out = "hACE2.box.byat.out"
json_box_out = "box.json"

prop = { 
    'execution_type' : 'check_only',
    'remove_tmp':False,
    'box_size_factor': 0.9
}

cmip(input_pdb_path=box_pdb_output,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     output_json_box_path=json_box_out,
     properties=prop)

In [ ]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(json_box_out, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view



In [ ]:
from biobb_cmip.cmip.ignore_residues import ignore_residues


inputPDB_MD_hACE2_ignored = "inputPDB_MD_hACE2_ignored.pdb"




prop = {
    'residue_list': "A:"
}

ignore_residues(input_cmip_pdb_path = box_pdb_output,
               output_cmip_pdb_path = inputPDB_MD_hACE2_ignored,
               properties = prop)

In [ ]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "hACE2.energies.log"
RBD_byat_out = "hACE2.energies.byat.out"

prop = { 
    'execution_type' : 'energy',
    'remove_tmp':False,
}

cmip(input_pdb_path=inputPDB_MD_hACE2_ignored,
     input_probe_pdb_path=cmipPDB_MD_hACE2,
     output_log_path=RBD_energies_log,
     output_byat_path=RBD_byat_out,
     input_json_box_path=json_box_out,
     properties=prop)

In [ ]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(RBD_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# ligands box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat


atom_list, energy_dict = get_energies_byat(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)



In [ ]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES'])],
       "layout": go.Layout(title="CMIP Interaction Potential", xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Interaction Energy Kcal/mol"))}

plotly.offline.iplot(fig)



<a id="titration"></a>
***
## PDB Titration
...
***
**Building Blocks** used:
 - [titration](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.titration) from **biobb_cmip.cmip.titration**
 - [cat_pdb](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

In [ ]:
from biobb_cmip.cmip.titration import titration

titration_input_pdb = data_folder + 'input2.pdb'
titration_output_pdb = 'output_tit.pdb'
titration_output_log = 'output_tit.log' 

prop = { 
    'num_positive_ions' : 5,
    'num_negative_ions' : 5,    
    'num_wats' : 20,
    'container_path': 'docker',
    'container_image': 'quay.io/biocontainers/cmip:2.7.0--h8c3ec31_0',
    'container_volume_path': '/inout'
}

titration(input_pdb_path=titration_input_pdb,
          output_pdb_path=titration_output_pdb,
          output_log_path=titration_output_log,
          properties=prop)

In [ ]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

cat_pdb_output = 'output_tit_all.pdb'

cat_pdb(input_structure1=titration_input_pdb,
        input_structure2=titration_output_pdb,
        output_structure_path=cat_pdb_output)

In [ ]:
view = nglview.show_structure_file(cat_pdb_output)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='spacefill', selection='water')
view.add_representation(repr_type='spacefill', selection='.Na', color='element')
view.add_representation(repr_type='spacefill', selection='.Cl', color='element')
view._remote_call('setSize', target='Widget', args=['','600px'])
view